In [ ]:
using AdaStress
import AdaStress.BlackBox

## Simulation

In [ ]:
using Distributions

In [ ]:
Base.@kwdef mutable struct Walk1D <: AdaStress.BlackBox
    t::Float64=0.0
    t_max::Float64=10.0
    x::Float64=0.0
    x_fail::Float64=10.0
    dist::Normal=Normal(0.0, 1.0)
end

function initialize!(sim::Walk1D)
    sim.t = 0.0
    sim.x = 0.0
end

function update!(sim::Walk1D)
    sim.t += 1.0
    a = rand(sim.dist)
    sim.x += a
    return logpdf(sim.dist, a) - logpdf(sim.dist, 0.0)
end

In [ ]:
AdaStress.reset!(sim::Walk1D) = initialize!(sim)
AdaStress.step!(sim::Walk1D) = update!(sim)
AdaStress.isterminal(sim::Walk1D) = sim.t >= sim.t_max
AdaStress.isevent(sim::Walk1D) = sim.x >= sim.x_fail
AdaStress.distance(sim::Walk1D) = max(sim.x_fail - sim.x, 0.0)

## Solver

In [ ]:
mcs = AdaStress.Solvers.MCS(num_steps=100_000)

In [ ]:
mdp = AdaStress.ASTMDP(Walk1D(); reward_bonus=100.0)

## Replay

In [ ]:
using Random

In [ ]:
Random.seed!(0)
sol = mcs(() -> mdp)

In [ ]:
AdaStress.reset!(mdp.sim)
for seed in collect(keys(sol))[end]
    AdaStress.@fix seed AdaStress.step!(mdp.sim)
end

In [ ]:
mdp.sim

In [ ]:
true